In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta

In [7]:
data_directory = "../Data/Study/"
hololenses = ['A/', 'B/']
subdirectory = "raw/"

In [3]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    result.reset_index(inplace=True, drop=True)
    return result

In [4]:
def calculate_differences(df):
    cleaned_df = clean_df(df)
    difference_df = cleaned_df.loc[1:, ['Movement', 'Time', 'GazeAngleX', 'GazeAngleY']]
    difference_df.reset_index(inplace=True, drop=True)
    difference_df['euclideanDifference'] = difference_df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY'])) - np.array((cleaned_df.loc[row.name, 'GazeAngleX'], cleaned_df.loc[row.name, 'GazeAngleY']))), axis=1)
    difference_df['temporalDifference'] = difference_df.apply(lambda row: (datetime.strptime(row['Time'], "%m/%d/%Y %H:%M:%S.%f") - datetime.strptime(cleaned_df.loc[row.name, 'Time'], "%m/%d/%Y %H:%M:%S.%f")).total_seconds(), axis=1)
    return difference_df

In [5]:
def calculate_precision():
    for hololens in hololenses:
        participantList = os.listdir(data_directory + hololens)
        for participant in participantList:
            print(participant)
            if participant ==  '.DS_Store':
                print("skipping calculation")
                continue
            if os.path.exists(data_directory + hololens + participant + '/precision/' + subdirectory):
                print("already calculated")
                continue
            fileList = os.listdir(data_directory + hololens + participant + '/' + subdirectory)
            for file in fileList:
                if '.csv' in file:
                    filePath = data_directory + hololens + participant + '/' + subdirectory + file
                    df = pd.read_csv(filePath)
                    difference_df = calculate_differences(df)
                    output_directory = data_directory + hololens + participant + '/precision/' + subdirectory
                    if not os.path.exists(output_directory):
                        os.makedirs(output_directory)
                    output_path = output_directory + file
                    difference_df.to_csv(output_path, index=False)

In [8]:
calculate_precision()

03
already calculated
04
already calculated
32
34
05
already calculated
02
.DS_Store
skipping calculation
18
already calculated
27
29
10
already calculated
19
already calculated
43
07
already calculated
36
09
already calculated
37
08
already calculated
01
already calculated
06
already calculated
39
46
41
49
47
25
50
35
33
.DS_Store
skipping calculation
20
already calculated
11
already calculated
16
already calculated
42
45
28
17
already calculated
26
21
already calculated
44
38
31
30
48
24
23
already calculated
15
already calculated
12
already calculated
40
13
already calculated
14
already calculated
22
already calculated
